## AI Agents


In [ ]:
%pip install -q smolagents[litellm] plotly geopandas shapely kaleido -q
%pip install -q smolagents

In [1]:
import pandas as pd

In [2]:
import numpy as np
import pandas as pd



import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
   
) -> float:
    """
    Calculate distance betwen two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
      

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
   

    # Format the results
    return round(distance, 2)


#print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))



def gen_rnd_normal(n_sample,mean_loc, dev_loc):
    """
    A method to return n_sample
    n_sample -  count of rows
    mean_loc -  the mean for each sample
    dev_loc -  the std for  each  column of samples

    return a matrix for locations
    """
 

    # Define means and standard deviations for two columns
    means = mean_loc  # Example means for two distributions
    std_devs = dev_loc # Example standard deviations
    
    # Generate a matrix with normally distributed columns
    rows = n_sample  # Number of rows
    cols = len(mean_loc)  # Number of columns
    
    matrix = np.column_stack([np.random.normal(mean, std_dev, size=rows) for mean, std_dev in zip(means, std_devs)])
    
    return matrix


In [3]:
mat_0 = gen_rnd_normal(10,[27.9475, -82.4584], [10,25])
mat_1 = gen_rnd_normal(10,[27.9475, -82.4584], [55,65])
mat = pd.DataFrame(np.concatenate((mat_0,mat_1), axis = 0))
patient_id = ['employee_'+str(i) for  i in range(mat.shape[0])]

mat['employee_id'] = patient_id

mat.columns = ["lat","long","employee_id"]
mat = mat.set_index('employee_id')

mat.to_json(orient = 'index') # return the geolocation as a string


'{"employee_0":{"lat":31.9904742408,"long":-122.9460445714},"employee_1":{"lat":19.4441879881,"long":-73.5153782701},"employee_2":{"lat":27.6665090247,"long":-98.1358210511},"employee_3":{"lat":21.6823063979,"long":-68.2981876722},"employee_4":{"lat":29.3196255548,"long":-100.268866647},"employee_5":{"lat":20.2014788068,"long":-45.9086180164},"employee_6":{"lat":28.7450445396,"long":-119.2986221988},"employee_7":{"lat":18.8964783743,"long":-132.2020320192},"employee_8":{"lat":25.0819869251,"long":-117.9508114796},"employee_9":{"lat":29.0087766251,"long":-101.3405392463},"employee_10":{"lat":142.2580151667,"long":-150.5910339445},"employee_11":{"lat":53.3770732647,"long":-83.8481024472},"employee_12":{"lat":120.2151109366,"long":-3.3236329754},"employee_13":{"lat":23.7339930768,"long":-119.6058104585},"employee_14":{"lat":-47.761429197,"long":-114.1716721951},"employee_15":{"lat":-21.4370558491,"long":-89.0062319093},"employee_16":{"lat":105.5939337824,"long":-2.1723873796},"employee_17

- **Reason/Reflect**
- **Plan**
- **Act**
- **Assess**

In [4]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel

# Initialize the model
model = HfApiModel()

# Create an agent with the DuckDuckGoSearchTool
#agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model)

# Run the agent with a sample query
#agent.run("How many seconds would it take for a leopard at full speed to run through Pont des Arts?")

In [5]:
import os
from PIL import Image
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel,VisitWebpageTool

# 
model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

In [6]:
agent = CodeAgent(
    model=model,
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas","json","requests"],
    max_steps=30,
)

In [7]:
task = """
Locate the geolocation data for all **Veterans Health Administration (VHA) facilities** in **California**.
Return the results in pandas *dataframe**  for structured data processing.
Save the  data frame CSV file as **VHA_CA.csv** in the current directory.

### Requirements:
- Include all **VHA facilities** in California.
- Format the CSV file correctly with appropriate headers for easy interpretation.
- Ensure data accuracy and completeness.
- Save the file properly in the specified location.

Optimize for efficiency and accuracy in retrieving and structuring location data.
"""



In [ ]:
agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Locate the geolocation data for all **Veterans Health Administration (VHA) facilities** in **California**.      │
│ Return the results in pandas *dataframe**  for structured data processing.                                      │
│ Save the  data frame CSV file as **VHA_CA.csv** in the current directory.                                       │
│                                                                                                                 │
│ ### Requirements:                                                                                               │
│ - Include all **VHA facilities** in California.                                                                 │
│ - Format the CSV file correctly with appropriate headers for easy interpretation.                               │
│ - Ensure data accuracy and completeness.                                                                        │
│ - Save the file properly in the specified location.                                                             │
│                                                                                                                 │
│ Optimize for efficiency and accuracy in retrieving and structuring location data.                               │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_results = web_search(query="Veterans Health Administration facilities in California geolocation data")    
  print(search_results)                                                                                            
                                                                                                                   
  url = "https://www.va.gov/oit/health-care-data/vha-facilities/"                                                  
  page_content = visit_webpage(url=url)                                                                            
  print(page_content)                                                                                              
                                                                                                                   
  import pandas as pd                                                                                              
                                                                                                                   
  # Download the dataset                                                                                           
  dataset_url = "https://data.va.gov/dataset/vha-facilities/resource/7a77e521-2255-40e8-9752-018a40f8676e"         
  vha_data = pd.read_csv(dataset_url)                                                                              
                                                                                                                   
  # Filter the DataFrame to include only facilities in California                                                  
  vha_ca = vha_data[vha_data['STATE'] == 'CA']                                                                     
                                                                                                                   
  # Save the filtered DataFrame as a CSV file                                                                      
  vha_ca.to_csv('VHA_CA.csv', index=False)                                                                         
                                                                                                                   
  # Print the first few rows of the DataFrame to verify the result                                                 
  print(vha_ca.head())                                                                                             
                                                                                                                   
  final_answer("VHA_CA.csv has been successfully created with geolocation data for all Veterans Health             
  Administration facilities in California.")                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'search_results = web_search(query="Veterans Health Administration facilities in 
California geolocation data")' due to: DuckDuckGoSearchException: https://lite.duckduckgo.com/lite/ 202 Ratelimit

[Step 1: Duration 10.23 seconds| Input tokens: 2,378 | Output tokens: 596]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Correct dataset URL                                                                                            
  dataset_url = "https://data.va.gov/dataset/vha-facilities/resource/7a77e521-2255-40e8-9752-018a40f8676e"         
                                                                                                                   
  # Download the dataset                                                                                           
  vha_data = pd.read_csv(dataset_url)                                                                              
                                                                                                                   
  # Filter the DataFrame to include only facilities in California                                                  
  vha_ca = vha_data[vha_data['STATE'] == 'CA']                                                                     
                                                                                                                   
  # Save the filtered DataFrame as a CSV file                                                                      
  vha_ca.to_csv('VHA_CA.csv', index=False)                                                                         
                                                                                                                   
  # Print the first few rows of the DataFrame to verify the result                                                 
  print(vha_ca.head())                                                                                             
                                                                                                                   
  final_answer("VHA_CA.csv has been successfully created with geolocation data for all Veterans Health             
  Administration facilities in California.")                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'vha_data = pd.read_csv(dataset_url)' due to: HTTPError: HTTP Error 404: Not Found

[Step 3: Duration 7.68 seconds| Input tokens: 12,671 | Output tokens: 1,537]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
url = "https://services2.arcgis.com/VFLAJVozK0rtzQmT/arcgis/rest/services/Veterans_Health_Administration_Medical_Facilities/FeatureServer/0/query?outFields=*&where=S_STATE%3D%27FL%27&outSR=4326&f=json"                             
page_content = visit_tool.forward(url)                                                                                
print(page_content) 
import json                                                                                                      
                                                                                                               
# Parse the JSON data                                                                                            
data = json.loads(page_content)                                                                                  
                                                                                                               
# Extract the features (facilities) from the data                                                                
facilities = data['features']                                                                                    
                                                                                                               
# Prepare a list to store the formatted facility data                                                            
formatted_facilities = []                                                                                        
                                                                                                               
# Iterate through each facility and extract relevant information                                                 
for facility in facilities:                                                                                      
  attributes = facility['attributes']                                                                          
  geometry = facility['geometry']                                                                              
  formatted_facility = {                                                                                       
      "Facility_Name": attributes['STA_NAME'],                                                                 
      "Facility_Type": attributes['S_ABBR'],                                                                   
      "Address": attributes['S_ADD2'],                                                                         
      "City": attributes['S_CITY'],                                                                            
      "State": attributes['S_STATE'],                                                                          
      "Zip": attributes['S_ZIP'],                                                                              
      "Zip4": attributes['S_ZIP4'],                                                                            
      "Latitude": geometry['y'],                                                                               
      "Longitude": geometry['x']                                                                               
  }                                                                                                            
  formatted_facilities.append(formatted_facility)                                                              
                                                                                                               
# Convert the list of facilities to JSON format                                                                  
facilities_json = json.dumps(formatted_facilities, indent=4)                                                     
                                                                                                               
# Save the JSON data to a file                                                                                   
with open('VHA_Fl.json', 'w') as json_file:                                                                      
  json_file.write(facilities_json, indent=4)                                                                             
                                                                                                               
print("Data saved to VHA_Fl.json")               

In [ ]:
url = "https://services2.arcgis.com/VFLAJVozK0rtzQmT/arcgis/rest/services/Veterans_Health_Administration_Medical_Facilities/FeatureServer/0/query?outFields=*&where=S_STATE%3D%27FL%27&outSR=4326&f=json"                             

visit_tool = VisitWebpageTool()

location_string = visit_tool.forward(url=url)



In [ ]:
employee_loc = mat.to_json(orient = 'index')

In [ ]:
task = f"""
You are an expert analyst specializing in **comprehensive report generation** based on **extensive web searches**. Your task is to identify the **nearest Veterans Health Administration (VHA) facility** for each employee while ensuring that the selected facility **accommodates extended work hours**.

### **Objectives**
- Conduct **optimized searches** using a **for loop** to process multiple queries efficiently.
- Match each employee with the **closest VHA facility** that meets their **specific operational requirements**.
- Accurately determine and **output the distance** between each employee and the nearest suitable facility.
- Use **employee geolocation data** and **facility geolocation data** to ensure precise matching.

### **Data Provided**
- **VHA Facility Locations** (structured data): `{location_string}`
- **Employee Geolocation Data** (structured data): `{employee_loc}`

### **Requirements**
- Implement an **efficient search strategy** to minimize redundancy and enhance accuracy.
- Prioritize facilities that **support extended work hours**, ensuring compatibility with employee needs.
- Optimize data processing to return **structured, well-organized results** for reporting purposes.
- return the distnace of each employee to the nearest facility in a pandas dataframe

**Deliver precise findings and streamline search strategies to enhance accessibility and operational efficiency.**
"""

In [ ]:
agent.run(task)